In [1]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from matplotlib.ticker import FormatStrFormatter

from sklearn.preprocessing import StandardScaler
import sklearn
import pickle

# from utils import plot_projections

from scipy.stats import kurtosis, skew

%matplotlib widget

In [2]:
def extract_speaker_num(f):
    
    substr = f.split('/')[7].split('_')
    
    speaker_str = re.split('(\d+)', substr[2])
    
    return int(speaker_str[1])

In [3]:
laser_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.wav')))
laser_files.sort()
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker10_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker10_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker10_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker10_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker10_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker11_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker11_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker11_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker11_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker11_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker12_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker12_002.wav', '/h

In [4]:
orig_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*wav')))
orig_files.sort()
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker10_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker10_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker10_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker10_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker10_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker11_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker11_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker11_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker11_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker11_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker12_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original

In [5]:
import splitfolders

In [8]:
# split original files into train and test

data_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/laser_injection_data'
output_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/'

splitfolders.ratio(data_dir, output=output_dir, seed=1337, ratio=(.7, 0, 0.3)) 

Copying files: 190 files [00:00, 5321.33 files/s]


In [ ]:
db1 = pywt.Wavelet('db1')

In [ ]:
sample_id = 10
laser_sample = laser_files[sample_id]
gen_sample = orig_files[sample_id]

laser_audio, sr = librosa.load(laser_sample, res_type='kaiser_fast')
gen_audio, sr = librosa.load(gen_sample, res_type='kaiser_fast')

stft_laser = librosa.stft(laser_audio, n_fft=256)
S_db_laser = librosa.amplitude_to_db(np.abs(stft_laser), ref=np.max)

stft_gen = librosa.stft(gen_audio, n_fft=256)
S_db_gen = librosa.amplitude_to_db(np.abs(stft_gen), ref=np.max)

In [ ]:
# fig, axes = plt.subplots(2, 1, figsize=(9,6))
# # ax = axes.flatten()

# img_laser = librosa.display.specshow(S_db_laser, x_axis='time', y_axis='log', sr=sr, ax=axes[0])
# # axes[0].set(title='Laser Audio Spectogram')
# # axes[0].label_outer()


# img_gen = librosa.display.specshow(S_db_gen, x_axis='time', y_axis='log', sr=sr, ax=axes[1])
# # axes[1].set(title='Genuine Audio Spectogram')
# # axes[1].label_outer()

# fig.colorbar(img_laser, ax=axes, format="%+2.f dB")

# plt.savefig("Spectogram_Comparison", dpi=300)

## Laser

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    
    audio_data = librosa.util.normalize(audio_data)
    
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf, ddof=1))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew_feat)
    laser_kurt_feat.append(kurt_feat)    
    
laser_sigma_feat = np.array(laser_sigma_feat)
laser_skew_feat = np.array(laser_skew_feat)
laser_kurt_feat = np.array(laser_kurt_feat)

In [ ]:
print(laser_sigma_feat.shape)
print(laser_skew_feat.shape)
print(laser_kurt_feat.shape)

## Original

In [ ]:
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    audio_data = librosa.util.normalize(audio_data)
    
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf, ddof=1))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew_feat)
    orig_kurt_feat.append(kurt_feat)    
    
orig_sigma_feat = np.array(orig_sigma_feat)
orig_skew_feat = np.array(orig_skew_feat)
orig_kurt_feat = np.array(orig_kurt_feat)

In [ ]:
print(orig_sigma_feat.shape)
print(orig_skew_feat.shape)
print(orig_kurt_feat.shape)

In [ ]:
# cluster plotting function for sigma, skew and kurtosis

def plot_clusters_skk(band_num):
    
#     plt.figure(figsize=(12,6))
    
    fig, ax = plt.subplots(1, 3, figsize=(18,6))
    
    
#     sigma vs skew
    ax[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[0].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[0].scatter(orig_sigma_feat[:,band_num], orig_skew_feat[:,band_num], marker='o', label="Original")
    ax[0].scatter(laser_sigma_feat[:,band_num], laser_skew_feat[:,band_num], marker='x', label="Laser")
    ax[0].set_xlabel("log-variance")
    ax[0].set_ylabel("skew")
#     plt.legend()
    
#     skew vs kurtosis
    ax[1].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[1].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[1].scatter(orig_skew_feat[:,band_num], orig_kurt_feat[:,band_num], marker='o', label="Original")
    ax[1].scatter(laser_skew_feat[:,band_num], laser_kurt_feat[:,band_num], marker='x', label="Laser")
    ax[1].set_xlabel("skew")
    ax[1].set_ylabel("kurtosis")
#     plt.legend()Spectogram_Comparison
     
#     kurtosis vs sigma
    ax[2].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[2].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax[2].scatter(orig_kurt_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    ax[2].scatter(laser_kurt_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    ax[2].set_xlabel("kurtosis")
    ax[2].set_ylabel("log-variance")
#     plt.legend()
    
    handles, labels = ax[2].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', prop={'size': 15})
    
#     plt.suptitle("Laser vs Original for {}th Band".format(band_num))
    
#     fig = plt.figure(figsize=(12,6))
# #     plt.figure(figsize=(20,6))
#     ax = fig.add_subplot(projection='3d')
#     ax.scatter(orig_sigma_feat[:,band_num], orig_skew_feat[:,band_num], orig_kurt_feat[:,band_num], marker='o', label="Original")
#     ax.scatter(laser_sigma_feat[:,band_num], laser_skew_feat[:,band_num], laser_kurt_feat[:,band_num], marker='x', label="Laser")
#     ax.set_xlabel('variance')
#     ax.set_ylabel('skew')
#     ax.set_zlabel('kurtosis')
#     ax.legend()

In [ ]:
aprox_detail_coeff = 0 
plot_clusters_skk(aprox_detail_coeff)

plt.savefig("Approx_band_analysis", dpi=300)

In [ ]:
aprox_detail_coeff = 1 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 2 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 3 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 4 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 5 
plot_clusters_skk(aprox_detail_coeff)

plt.savefig("Detail_band_analysis", dpi=300)

## SVM with Linear Kernel

In [ ]:
import sklearn

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

In [ ]:
X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=1)
laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape

In [ ]:
X_orig = np.concatenate((orig_sigma_feat, orig_skew_feat, orig_kurt_feat), axis=1)
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

In [ ]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [ ]:
X = np.concatenate((X_laser, X_orig))
X.shape

In [ ]:
y = np.concatenate((laser_label, orig_label))
y.shape

In [ ]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
X_train.shape

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("accuracy = ", accuracy_score(y_test, y_pred))

In [ ]:
# save the model to disk

model_dir = 'models/'
if not os.path.exists(model_dir):
    
    os.makedirs(model_dir)
    print("The new directory is created!")
    
filename = model_dir + 'svm_model_male_female_speaker_norm.sav'
pickle.dump(clf, open(filename, 'wb'))

## with Log sigma

In [ ]:
import math

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    audio_data = librosa.util.normalize(audio_data)
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf
        
        sigma_feat.append(np.log(np.var(cf)))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew_feat)
    laser_kurt_feat.append(kurt_feat)    
    
laser_sigma_feat = np.array(laser_sigma_feat)
laser_skew_feat = np.array(laser_skew_feat)
laser_kurt_feat = np.array(laser_kurt_feat)

In [ ]:
print(laser_sigma_feat.shape)
print(laser_skew_feat.shape)
print(laser_kurt_feat.shape)

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    audio_data = librosa.util.normalize(audio_data)
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf
        
        sigma_feat.append(np.log(np.var(cf)))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew_feat)
    orig_kurt_feat.append(kurt_feat)    
    
orig_sigma_feat = np.array(orig_sigma_feat)
orig_skew_feat = np.array(orig_skew_feat)
orig_kurt_feat = np.array(orig_kurt_feat)

In [ ]:
aprox_detail_coeff = 0 
plot_clusters_skk(aprox_detail_coeff)

plt.savefig("Approx_band_analysis_logvar", dpi=300)

In [ ]:
aprox_detail_coeff = 1 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 2 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 3 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 4 
plot_clusters_skk(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 5 
plot_clusters_skk(aprox_detail_coeff)

plt.savefig("Detail_band_analysis_logvar", dpi=300)

## log-SVM with Linear Kernel

In [ ]:
import sklearn

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

In [ ]:
X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=1)
laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape

In [ ]:
X_orig = np.concatenate((orig_sigma_feat, orig_skew_feat, orig_kurt_feat), axis=1)
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

In [ ]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [ ]:
X = np.concatenate((X_laser, X_orig))
X.shape

In [ ]:
y = np.concatenate((laser_label, orig_label))
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("accuracy = ", accuracy_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

plt.savefig("ConfusionMatrix", dpi=300)

In [ ]:
# save the model to disk

model_dir = 'models/'
if not os.path.exists(model_dir):
    
    os.makedirs(model_dir)
    print("The new directory is created!")
    
filename = model_dir + 'svm_model_log_sigma_male_female_speaker_norm.sav'
pickle.dump(clf, open(filename, 'wb'))

## What if we only use variance as feature

In [ ]:
# X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=1)
X_laser = laser_sigma_feat
laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape

In [ ]:
# X_orig = np.concatenate((orig_sigma_feat, orig_skew_feat, orig_kurt_feat), axis=1)
X_orig = orig_sigma_feat
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

In [ ]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [ ]:
X = np.concatenate((X_laser, X_orig))
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("accuracy = ", accuracy_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

plt.savefig("ConfusionMatrix_variance", dpi=300)

## What if we just use first approximation band ?

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
#     sigma_feat = []
#     skew_feat = []
#     kurt_feat = []
    
    sigma_feat = np.log(np.var(coeffs[0]))
    
#     skew_feat.append(skew(cf))
#     kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew(coeffs[0]))
    laser_kurt_feat.append(kurtosis(coeffs[0]))    
    
# laser_sigma_feat = np.array(laser_sigma_feat)
# laser_skew_feat = np.array(laser_skew_feat)
# laser_kurt_feat = np.array(laser_kurt_feat)

In [ ]:
# print(laser_sigma_feat.shape)
print(laser_sigma_feat)

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    sigma_feat = np.log(np.var(coeffs[0]))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew(coeffs[0]))
    orig_kurt_feat.append(kurtosis(coeffs[0]))    
    
# orig_sigma_feat = np.array(orig_sigma_feat)
# orig_skew_feat = np.array(orig_skew_feat)
# orig_kurt_feat = np.array(orig_kurt_feat)

In [ ]:
X_laser = [laser_sigma_feat]
X_laser.append(laser_skew_feat)
X_laser.append(laser_kurt_feat)

X_laser = np.array(X_laser).transpose()

# print(X_laser)
# X_laser = np.array(X_laser.append(laser_kurt_feat))
# X_laser = laser_sigma_feat.append(laser_skew_feat).append(laser_kurt_feat)
# X_laser = np.array(X_laser)
# X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=0)
# X_laser = np.array([[laser_sigma_feat], [laser_skew_feat], [laser_kurt_feat]])

laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape
# X_laser = laser_sigma_feat
# laser_label = np.array(['laser']*X_laser.shape[0])
# print(laser_label)
# print(laser_label.shape)
# X_laser.shape

In [ ]:
X_orig = [orig_sigma_feat]
X_orig.append(orig_skew_feat)
X_orig.append(orig_kurt_feat)

X_orig = np.array(X_orig).transpose()

# print(X_orig)

# X_orig = orig_sigma_feat
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

In [ ]:
# print(X_laser)

In [ ]:
# print(X_orig)

In [ ]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [ ]:
X = np.concatenate((X_laser, X_orig))
X.shape
print(X)

In [ ]:
y = np.concatenate((laser_label, orig_label))
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("accuracy = ", accuracy_score(y_test, y_pred))

## What if we just use variance of first approximation band ?

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
# laser_skew_feat = []
# laser_kurt_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
#     sigma_feat = []
#     skew_feat = []
#     kurt_feat = []
    
    sigma_feat = np.log(np.var(coeffs[0]))
    
#     skew_feat.append(skew(cf))
#     kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
#     laser_skew_feat.append(skew(coeffs[0]))
#     laser_kurt_feat.append(kurtosis(coeffs[0]))    
    
# laser_sigma_feat = np.array(laser_sigma_feat)
# laser_skew_feat = np.array(laser_skew_feat)
# laser_kurt_feat = np.array(laser_kurt_feat)

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
# orig_skew_feat = []
# orig_kurt_feat = []

for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    sigma_feat = np.log(np.var(coeffs[0]))
        
    orig_sigma_feat.append(sigma_feat)
#     orig_skew_feat.append(skew(coeffs[0]))
#     orig_kurt_feat.append(kurtosis(coeffs[0]))    
    
# orig_sigma_feat = np.array(orig_sigma_feat)
# orig_skew_feat = np.array(orig_skew_feat)
# orig_kurt_feat = np.array(orig_kurt_feat)

In [ ]:
X_laser = [laser_sigma_feat]
# X_laser.append(laser_skew_feat)
# X_laser.append(laser_kurt_feat)

X_laser = np.array(X_laser).transpose()

# print(X_laser)
# X_laser = np.array(X_laser.append(laser_kurt_feat))
# X_laser = laser_sigma_feat.append(laser_skew_feat).append(laser_kurt_feat)
# X_laser = np.array(X_laser)
# X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=0)
# X_laser = np.array([[laser_sigma_feat], [laser_skew_feat], [laser_kurt_feat]])

laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape
# X_laser = laser_sigma_feat
# laser_label = np.array(['laser']*X_laser.shape[0])
# print(laser_label)
# print(laser_label.shape)
# X_laser.shape

In [ ]:
X_orig = [orig_sigma_feat]
# X_orig.append(orig_skew_feat)
# X_orig.append(orig_kurt_feat)

X_orig = np.array(X_orig).transpose()

# print(X_orig)

# X_orig = orig_sigma_feat
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

In [ ]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [ ]:
X = np.concatenate((X_laser, X_orig))
X.shape
# print(X)

In [ ]:
y = np.concatenate((laser_label, orig_label))
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("accuracy = ", accuracy_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

plt.savefig("ConfusionMatrix_variance_CA5", dpi=300)